# Setup
* Installing the relevant libraries, 
* Checking if GPU is available,
* Checking and setting up device.

In [1]:
!pip install transformers
!pip install colabcode
!pip install fastapi

In [ ]:
!nvidia-smi

In [2]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'

## Code generation example:

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Salesforce/codegen-350M-mono")
model = AutoModelForCausalLM.from_pretrained("Salesforce/codegen-350M-mono").to(device)

In [ ]:
text = "def standardize_array(arr):"
tokenized = tokenizer(text, return_tensors="pt").to(device)
input_ids = tokenized.input_ids

In [ ]:
generated_ids = model.generate(input_ids, max_length=128,
                               pad_token_id=tokenizer.eos_token_id,
                               penalty_alpha=0.4,
                               top_k=3,
                               temperature=0.8)

print(tokenizer.decode(generated_ids[0].cpu().tolist(), skip_special_tokens=True))

## Code summarization example:

In [4]:
from transformers import RobertaTokenizer, T5ForConditionalGeneration

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained('Salesforce/codet5-base-multi-sum')
model = T5ForConditionalGeneration.from_pretrained('Salesforce/codet5-base-multi-sum').to(device)

In [ ]:
text = """def svg_to_image(string, size=None):
if isinstance(string, unicode):
    string = string.encode('utf-8')
    renderer = QtSvg.QSvgRenderer(QtCore.QByteArray(string))
if not renderer.isValid():
    raise ValueError('Invalid SVG data.')
if size is None:
    size = renderer.defaultSize()
    image = QtGui.QImage(size, QtGui.QImage.Format_ARGB32)
    painter = QtGui.QPainter(image)
    renderer.render(painter)
return image"""
input_ids = tokenizer(text, return_tensors="pt").input_ids.to(device)

In [ ]:
generated_ids = model.generate(input_ids, max_length=20,
                               pad_token_id=tokenizer.eos_token_id,
                               penalty_alpha=0.6,
                               top_k=4,
                               temperature=0.8)
print(tokenizer.decode(generated_ids[0], skip_special_tokens=True))

Converts a SVG string to a QImage.


# Server
* Using ColabCode a python package that simplifies exposing and running a server in colab (Ngrok + Uvicorn),
* Using FastAPI to handle POST requests.

In [5]:
from colabcode import ColabCode
from fastapi import FastAPI
from pydantic import BaseModel

In [6]:
cc = ColabCode(port=12000, code=False)

In [10]:
app = FastAPI(title="NLP_Code", description="NLP code completition and summarization", version="1.0")

In [11]:
class Item(BaseModel):
    text: str

model_sum = None
token_sum = None
model_comp = None
token_comp = None
device = None

@app.on_event("startup")
def load_model():
    global model_sum
    global token_sum
    global model_comp
    global token_comp
    global device

    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    token_comp = AutoTokenizer.from_pretrained("Salesforce/codegen-350M-mono")
    model_comp = AutoModelForCausalLM.from_pretrained("Salesforce/codegen-350M-mono").to(device)

    token_sum = RobertaTokenizer.from_pretrained('Salesforce/codet5-base-multi-sum')
    model_sum = T5ForConditionalGeneration.from_pretrained('Salesforce/codet5-base-multi-sum').to(device)


@app.post("/completion")
async def get_completion(input: Item):
    try:
        input_ids = token_comp(input.text, return_tensors="pt").input_ids.to(device)

        generated_ids = model_comp.generate(input_ids, max_length=128,
                               pad_token_id=token_comp.eos_token_id,
                               penalty_alpha=0.4,
                               top_k=3,
                               temperature=0.8)

        result = token_comp.decode(generated_ids[0].cpu().tolist(), skip_special_tokens=True)
        return {"completion": result}
    except:
        my_logger.error("Something went wrong!")
        return {"completion": "error"}

@app.post("/summarization")
async def get_summary(input: Item):
    try:
        input_ids = token_sum(input.text, return_tensors="pt").input_ids.to(device)

        generated_ids = model_sum.generate(input_ids, max_length=40,
                               pad_token_id=token_sum.eos_token_id,
                               penalty_alpha=0.6,
                               top_k=5,
                               temperature=0.8)

        result = token_sum.decode(generated_ids[0].cpu().tolist(), skip_special_tokens=True)
        return {"summary": result}
    except:
        my_logger.error("Something went wrong!")
        return {"completion": "error"}

In [12]:
cc.run_app(app=app)

Public URL: NgrokTunnel: "https://55e1-34-66-229-37.ngrok.io" -> "http://localhost:12000"


INFO:     Started server process [1067]
INFO:uvicorn.error:Started server process [1067]
INFO:     Waiting for application startup.
INFO:uvicorn.error:Waiting for application startup.
INFO:     Application startup complete.
INFO:uvicorn.error:Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:12000 (Press CTRL+C to quit)
INFO:uvicorn.error:Uvicorn running on http://127.0.0.1:12000 (Press CTRL+C to quit)


INFO:     89.143.45.194:0 - "POST /summarization HTTP/1.1" 200 OK


/usr/local/lib/python3.8/dist-packages/transformers/models/codegen/modeling_codegen.py:166: UserWarning: where received a uint8 condition tensor. This behavior is deprecated and will be removed in a future version of PyTorch. Use a boolean condition instead. (Triggered internally at ../aten/src/ATen/native/TensorCompare.cpp:413.)
  attn_weights = torch.where(causal_mask, attn_weights, mask_value)


INFO:     89.143.45.194:0 - "POST /completion HTTP/1.1" 200 OK
INFO:     89.143.45.194:0 - "POST /completion HTTP/1.1" 200 OK
INFO:     89.143.45.194:0 - "POST /summarization HTTP/1.1" 200 OK
INFO:     89.143.45.194:0 - "POST /summarization HTTP/1.1" 200 OK
INFO:     89.143.45.194:0 - "POST /completion HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:uvicorn.error:Shutting down
INFO:     Waiting for application shutdown.
INFO:uvicorn.error:Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:uvicorn.error:Application shutdown complete.
INFO:     Finished server process [1067]
INFO:uvicorn.error:Finished server process [1067]
